$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# 3. Measuring a Bicycle Wheel's Inertia

This notebook introduces the concept of using vibratory characteristics to estimate parameters of an existing system. It discusses how vibrations can be measured and how these measurements might relate to parameters of interest, such as the inertia of a bicycle wheel.

After the completion of this assignment students will be able to:

-   describe different methods of measuring vibrations
-   choose appropriate sensors and sensor placement
-   visualize the vibrational measurements
-   use curve fitting to estimate the period of oscillation
-   understand the concept of natural frequency and its relationship to mass/inertia and stiffness
-   state two of the three fundamental characteristics that govern vibration (mass/inertia and stiffness)
-   use frequency domain techniques to characterize a system's behavior

## Inertia and Vibration

One of the fundamental properties that affects how systems vibrate is the inertia of a system. Inertia is colloquially defined as the *tendency to do nothing or to remain unchanged*. A vibrating system is changing with respect to time, thus we may infer that inertia will try to prevent the system from changing. There are two specific types of inertia that we will address: mass which is a resistance to linear motion and moments of inertia which is a resistenace to angular motion. The moment of inertia can also be thought a descriptor for the distribution of mass in a system.

Most people are familiar with a bicycle wheel, so we will look into the inertial characteristics of bicycle wheels and how this relates to vibration. A bicycle wheel is generally symmeetric about the plane defined by the circular rim and tire in addition to being symmetric about any radial plane normal to the wheel plane. Recalling from dynamics, this means that the momement of inertia will have three priniciapl moments of inertia, two of which are equal. The inertia about the axis of the wheel is the rotational inertia (resists the wheel rolling) and the inertia about any radial axis resists motions like turning the handlebars, etc.


We demonstrated in the previous lesson that the inertia of the book affected the frequency of oscillation.

A common way to measure inertia is to use a spring with a known linear force/displacement ratio, the spring constant or stiffness, to resist motion. If we attached a torsional spring along a radial line of the bicycle wheel to the wheel and a fixed ceiling. If you twist the wheel about the axis of the torsional spring it will start vibrating.

With a spring and a mass/inertia you can cause vibration. The spring constant is the second fundamental property of vibration.

The video belows shows a bicycle wheel that is attached to the ceiling by a thin steel rod. The rod's axis passing through the center of mass of the wheel. When the bicycle wheel is given an initial angular displacement from the equilibrium, it vibrates.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('uuc1_RTYZLw', width=640, height=480)

A free body diagram can be sketched of this system, see below:

<img src="fig/03/bicycle-wheel-radial-inertia-measurement-fbd.png" alt="image" width="400" />

**Exercise**

This system is slightly different than the book and cup system in the previous lesson. There are two fundamental properties of the system that make this it vibrate. What are these two things?

## Data

During the experiment shown in the above video I recorded some important constants about the system and measured the angular rate of the system about the rod's axis during rotation.

A rate gyro is a micro electromechanical device that outputs a voltage proportional to the angular rate 

The data from the measurement can be loaded into a Panda's `Series` with Panda's `read_csv()` function. The time index is in seconds and the angular velocity is in radians per second.

In [ ]:
import pandas as pd
gyro_reading = pd.read_csv('data/03/bicycle-wheel-radial-inertia-rate-gyro-measurement.csv', index_col='time')

In [ ]:
gyro_reading.head()

In [ ]:
gyro_reading.tail()

This can then be plotted to see what the signal looks like, use the dot style to show the individual measurements:

In [ ]:
%matplotlib notebook

In [ ]:
gyro_reading.plot(style='.');

**Exercise**

Use your period estimation function from the previous lesson to try to get an estimate of the period of this oscillation.

In [ ]:
from resonance.functions import estimate_period

In [ ]:
estimate_period(gyro_reading.index, gyro_reading.angular_rate)

# Simulating the system

There is a system in the `resonance` package that represents the bicycle wheel and torsion rod system:

In [ ]:
from resonance.linear_systems import BicycleWheelRadialInertiaSystem

In [ ]:
sys = BicycleWheelRadialInertiaSystem()

Note that the constants are not yet set:

In [ ]:
sys.constants

**Exercise**

Calculate the stiffness of the rod based on your knowledge of torsional elastic mechanics. Here are some of the torsion rod's geometry and material properties:

-   Length of the torsion rod, $l$ : 1.05 m
-   Diameter of the torsion rod, $d$ : 6.35 mm
-   Modulus of Rigidity of steel, $G$ : 77 GPa

Set the system's rod stiffness to the value you calculate.

In [ ]:
import numpy as np

In [ ]:
rod_stiffness = 77.0E9 * np.pi * 0.00635**4 / 32 / 1.05  # N/m
rod_stiffness

In [ ]:
sys.constants['rod_stiffness'] = rod_stiffness

In [ ]:
sys.constants

**Exercise**

As a starting estimate of the inertia of the bicycle wheel, use the axial moment of inertia of an infinitely thin hoop of mass, $m$. See Wikipedia's [List of moments of inertia](https://en.wikipedia.org/wiki/List_of_moments_of_inertia) for the equation. The bicycle wheel used in the data collection had these values:

-   Outer radius of the bicycle wheel, $r$ : 0.336 m
-   Mass of the bicycle wheel, $m$ : 1.55 kg

Set the axial inertia value in the system to the value you calculate.

In [ ]:
axial_inertia = 1.55 * 0.336**2 / 4.0 # kg m**2
sys.constants['axial_inertia'] = axial_inertia
sys.constants

**Exercise**

Use the above constants and simulate the system with an initial angular velocity that matches the initial measured angular velocity in the measurements. You can set the initial angular velocity by accessing the `System.speeds` dictionary. Calculate the period of oscillation of this simulation and see if it is similar to the data. One way to select a single value from a Pandas `DataFrame` is to use the `.loc[]` notation. This will allow you to select a row by it's index.

In [ ]:
sys.speeds['torsion_angle_vel'] = gyro_reading.loc[0.0]['angular_rate']
sys.speeds

In [ ]:
duration = gyro_reading.index[-1] - gyro_reading.index[0]
duration

In [ ]:
traj = sys.free_response(duration)

In [ ]:
traj.plot(subplots=True);

In [ ]:
sys.period()

## Interactive Plots

You should see that the computational system does not quite predict the actual period of the oscillatoin. One useful tool for interactively changing the constants of the system is a Jupyter "widget". Widgets let you create sliders, drop downs, value entry boxes, etc to interactively change the results of the simulation. Below is an example of how you can create a widget that updates a plot interactively. In this case we'd like to adjust the inertia value until the period of oscillation of the simulation matches the period of the actual data.

The first step is to create a plot of the data, as we did above, but make sure to assign the axis that command creates to a variable `ax` so that we can add more information to the plot. The second step is to define a function which has a input that you want to adjust interactively, in our case the axial inertia. This function should compute the new simulation trajectory, set the values for the simulation line with the new data, and finally redraw the figure.

In [ ]:
import matplotlib.pyplot as plt

ax = gyro_reading.plot(style='.')
line = ax.plot(traj.index, traj.torsion_angle_vel)
ax.set_ylabel('Angular Rate [rad/s]')
ax.set_xlabel('Time [s]')
ax.legend(['Measured', 'Simulation'], loc=1)

def plot_trajectory_comparison(axial_inertia=0.044):
   sys.constants['axial_inertia'] = axial_inertia  # set the new inertia value
   traj = sys.free_response(duration, sample_rate=300)  # simulate the system with new value
   line[0].set_data(traj.index, traj.torsion_angle_vel)  # set the x and y data of the simulation line to new data
   plt.gcf().canvas.draw()  # redraw the figure with the updated line

# call the function to make the initial plot
plot_trajectory_comparison()

Now the fun part. You can use the `interact` function from the `ipywidgets` package to make the plot interactive. You pass in the function that does the plot updating and then a range for the slider for the axial inertia as `(smallest_value, largest_value, step_size)`. This will create a slider that updates the plot as you change the value.

In [ ]:
from ipywidgets import interact
widget = interact(plot_trajectory_comparison, axial_inertia=(0.0, 0.2, 0.001));

Here is a way to get to the value that the widget is set too (it's a bit obtuse, but we'll show different methods later):

In [ ]:
widget.widget.children[0].value

# How does inertia relate to the period?

By now it should be pretty clear that there is a relationship between the inertia of the system and the period of oscillation. It would be nice to plot the frequency versus the change in inertia to try and determine what the relationship is. Say we want to check a range of inertia values from X to Y, we can create those values with:

In [ ]:
import numpy as np
inertias = np.linspace(0.001, 0.2, num=50)

## Python For Loops

Instead of typing the simulation code out for each of the 100 inertia values, we can use a loop to iterate through each value and save some typing. Python loops are constructed with the `for` command and, in the simplest case, the `in` command. Here is an exampel that iterates through the inertia values, computes what the hoop radius would be in the mass was fixed, and prints the value of the radius on each iteration:

In [ ]:
for inertia in inertias:
    radius = np.sqrt(2 * inertia / 1.55)
    print(radius)

It is also often useful to store the computed values in a list or array as the loop iterates so you can use the values in future computations. Here is a way to store the values in a Python list and then convert the list to a NumPy array.

In [ ]:
radii = []
for inertia in inertias:
    radius = np.sqrt(2 * inertia / 1.55)
    radii.append(radius)
radii = np.array(radii)
radii

If you want to work with the NumPy array up front you can create an empty NumPy array and then fill it by using the correct index value. This index value can be exposed using the `enumerate()` function. For example:

In [ ]:
radii = np.empty_like(inertias)
radii

In [ ]:
for i, inertia in enumerate(inertias):
    radius = np.sqrt(2 * inertia / 1.55)
    radii[i] = radius
radii

It is also worth noting that NumPy provides *vectorized* functions and that loops are not needed for the above example, for example:

In [ ]:
radii = np.sqrt(2 * inertias / 1.55)
radii

It is a good idea to use vectorized functions instead of loops when working with arrays if you can, because they are much faster:

In [ ]:
%%timeit

radii = []
for inertia in inertias:
    radius = np.sqrt(2 * inertia / 1.55)
    radii.append(radius)
radii = np.array(radii)

In [ ]:
%%timeit

np.sqrt(2.0 * inertias / 1.55)

**Exercise**

Use a loop to construct a list of periods for different inertia values. After you have both arrays, plot the inertias on the $X$ axis and the frequencies on the $Y$ axis. Is there any functional relationship that describes the relationship between the variables?

In [ ]:
periods = []
for inertia in inertias:
    sys.constants['axial_inertia'] = inertia
    periods.append(sys.period())

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(inertias, periods)
ax.set_xlabel('Inertia [$kg \cdot m^2$]')
ax.set_ylabel('Period [s]');

In [ ]:
plt.figure()
x = np.linspace(0, 0.2, 100)
plt.plot(x, np.sqrt(x))
plt.xlabel('$x$')
plt.ylabel('$\sqrt{x}$');

It turns out that the period of oscillationo, $T$, is proportional to the square root of the oscillating inertia.

$$ T \propto \sqrt{I} $$

and more precisely the proportionality coefficient ends up being:

$$ \frac{2\pi}{\sqrt{k}} $$

we will discuss why this is the case when we get to modeling in the course.

# Curve Fitting

Above, we interactive adjusted the inertia such that the simulation's period matched the measured data.

The Python package `scipy` provides a very conveinent function.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
 1.0 / 0.44 * np.pi * 2 # cycles / second * 2 * pi radaisn / cycle

In [ ]:
def cos_func(times, frequency, amplitude):
    return amplitude * np.cos(frequency * times)

In [ ]:
popt, pcov = curve_fit(cos_func, gyro_reading.index, gyro_reading.angular_rate,
                       p0=(1.0 / 0.44 * np.pi * 2, 3.0))

In [ ]:
popt

In [ ]:
ax = gyro_reading.plot(style='.')
ax.plot(gyro_reading.index, cos_func(gyro_reading.index, *popt))

In [ ]:
1.0 / popt[0] * np.pi * 2.0

In [ ]:
def decaying_sinusoid(t, a, w, p, m):
    return np.exp(m * t) * a * np.sin(w * t + p)

In [ ]:
popt, pcov = curve_fit(decaying_sinusoid, gyro_reading.index, gyro_reading.angular_rate,
          p0=(3.0, 1.0 / 0.44 * np.pi * 2, 0.0, -0.0002))

In [ ]:
ax = gyro_reading.plot(style='.')
ax.plot(gyro_reading.index, decaying_sinusoid(gyro_reading.index, *popt));

In [ ]:
1.0 / popt[1] * np.pi * 2.0

mP = 4.65+/-0.01 dP = 0.03009+/-0.00001, 0.03010+/-0.00001, 0.03012+/-0.00001 lP = 0.8355+/-0.001 period of calib rod oscillation Tp = 0.9561002971327414

Ip = mP / 12 \* (3\*rp^2 + lp^2) Ip = 0.2707614811040625 stiffness of rod k = 4 \* Ip \* np.pi\**2 / Tp*\*2 k = 11.693370530226998

k = G Jp / l Jp = np.pi \* d\**4 / 32 \# polar second moment of are of torsion rod l = G* Jp / k = G \* np.pi \* d\*\*4 / 32 / k if rod is 1/4" then length of torsion rod is 1.0511042914686415 meters

IFxx = 0.0524475128396 kg m\*\*2 IFyy = 0.0983720589324

k \* T\**2 / 4 / np.pi*\*2 = Ip Front wheel compount pendulum length lF = 0.2957195+/-0.00008

In [ ]:
mP = 4.65
dP = 0.03009
lP = 0.8355
rP = dP / 2.0
TP = 0.9561002971327414
rod_stiffness = mP * (3 * rP**2 + lP**2) * np.pi**2 / 3 / TP**2
rod_stiffness

In [ ]:
G = 77.0E9
r = 0.00635 / 2.0
l = G * np.pi * r**4 / 2 / rod_stiffness
l

In [ ]:
G * np.pi * r**4 / 2.0 / l

In [ ]:
# moment of inertia about axis computed from BicycleParameters
IFyy =  0.0983720589324  # kg m**2
mF = 1.55  # kg
rF = 0.335572561302  # m

In [ ]:
# the simplification under predicts the inertia by a bit
IFyy_simple = mF * rF**2 / 2.0
IFyy_simple

In [ ]:
rF_simple = np.sqrt(2.0 * IFyy / mF)
rF_simple